1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [7]:
import pandas as pd

Наши новости

In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

--2023-06-30 10:10:12--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 172.217.203.113, 172.217.203.139, 172.217.203.101, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4d33fj4jd4d04di8mb3pltd69fmn1bhi/1688119800000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=bd4631b9-bd22-4871-bcfe-dbbe72f89a31 [following]
--2023-06-30 10:10:12--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4d33fj4jd4d04di8mb3pltd69fmn1bhi/1688119800000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download&uuid=bd4631b9-bd22-4871-bcfe-dbbe72f89a31
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 74.125.134.1

In [8]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [9]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [10]:
!pip install razdel pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 63.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=4649df487d5ec148f56f8d4e5925a4ea556de531809561c4703b1fffe5c3ffe7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [11]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [14]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2023-06-30 10:14:21--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 74.125.196.101, 74.125.196.102, 74.125.196.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.196.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ut6v2kn1g14uq178qhvaub37rcogpqob/1688120025000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=6e4168d6-2e1a-441e-ba82-367abc24a9b2 [following]
--2023-06-30 10:14:22--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ut6v2kn1g14uq178qhvaub37rcogpqob/1688120025000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=6e4168d6-2e1a-441e-ba82-367abc24a9b2
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 74.125.134.132, 2607:

In [15]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]

stopword_ru += additional_stopwords
len(stopword_ru)

776

In [16]:
def clean_text(text):
    '''
    очистка текста

    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)

    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]

    return words_lem_without_stopwords

In [17]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/27000 [00:00<?, ?it/s]<ipython-input-16-898b573b2830>:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|██████████| 27000/27000 [00:39<00:00, 679.59it/s] 


In [18]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [19]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 27000/27000 [04:03<00:00, 110.88it/s]


А теперь в 3 строчки обучим нашу модель

In [20]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [21]:
N_topic = 20  # можно было менять

In [22]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [23]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [24]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [25]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(1, 0.086375214),
 (4, 0.41972595),
 (11, 0.2952916),
 (17, 0.023492465),
 (19, 0.15479995)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [26]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: банк российский санкция правительство министр россия рф
topic_1: снижение космос снизить организм иран риск граница
topic_2: городской мышь ремонт парк сезон гостиница туризм
topic_3: земля ребёнок мозг врач район произойти помощь
topic_4: исследование первый всё день большой новый жизнь
topic_5: гражданин рак болезнь палата россиянин фронт регистрация
topic_6: погибнуть километр живой рекорд дым герман толчок
topic_7: температура прогноз погода знаменитый градус треть дождь
topic_8: остров операция экипаж восток аэропорт участок активность
topic_9: украина украинский киев пётр народный русский украинец
topic_10: взрыв запустить запуск полицейский эксперимент соцсеть ii
topic_11: китай налог китайский конкурс корея южный золото
topic_12: выяснить население al боль белый страдать общество
topic_13: фонд рубль сотрудник тыс уголовный размер млн
topic_14: россия российский путин государство глава военный территория
topic_15: россия млрд проект рост цена рубль рынок
topic_16: сша 

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [27]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [28]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.71873099, 0.        , 0.08637829, 0.0367497 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03281811, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03191979, 0.08638507, 0.        ])

In [29]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.718728,0.000000,0.086378,0.036787,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.032820,0.0,0.000000,0.0,0.0,0.0,0.03192,0.086349,0.000000
1,4896,0.000000,0.165937,0.000000,0.000000,0.000000,0.0,0.018571,0.0,0.0,...,0.229561,0.395953,0.0,0.170234,0.0,0.0,0.0,0.00000,0.000000,0.000000
2,4897,0.000000,0.086368,0.000000,0.000000,0.419587,0.0,0.000000,0.0,0.0,...,0.000000,0.295330,0.0,0.000000,0.0,0.0,0.0,0.02360,0.000000,0.154801
3,4898,0.072295,0.000000,0.058227,0.000000,0.684322,0.0,0.000000,0.0,0.0,...,0.000000,0.101614,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000
4,4899,0.816075,0.000000,0.157428,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [30]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [31]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [32]:
doc_dict[293672]

array([0.        , 0.        , 0.11811846, 0.        , 0.10314247,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.40770102, 0.11753933, 0.17258082, 0.06146269, 0.        ])

In [33]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [34]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.08069596, 0.00232509, 0.0061792 , 0.        , 0.17036039,
       0.01255988, 0.        , 0.0186161 , 0.0068239 , 0.00253607,
       0.        , 0.        , 0.        , 0.11342871, 0.17083431,
       0.17535116, 0.02371131, 0.05240769, 0.1488493 , 0.        ])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [35]:
FUNC = np.mean

In [36]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.007214,0.049746,0.063232,0.213117,0.002548,0.000000,0.002989,0.000000,...,0.006121,0.000000,0.007045,0.042672,0.098515,0.124398,0.044201,0.060642,0.115349,0.022772
1,u108690,0.018730,0.000000,0.000000,0.070961,0.237472,0.004851,0.000000,0.000000,0.004799,...,0.001898,0.011698,0.007499,0.117693,0.106447,0.113055,0.088997,0.006221,0.095248,0.003433
2,u108339,0.090183,0.016986,0.016728,0.137240,0.157661,0.000000,0.004503,0.026918,0.010828,...,0.000000,0.002067,0.001869,0.104995,0.153738,0.095754,0.038900,0.034494,0.093090,0.000000


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [37]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2023-06-30 10:33:44--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Resolving drive.google.com (drive.google.com)... 142.251.107.138, 142.251.107.101, 142.251.107.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.107.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/okd4o04m7ad65fn4tin87ib4ahbf492m/1688121225000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download&uuid=07de3a47-fcdb-4e53-b97a-19818b579f36 [following]
--2023-06-30 10:33:45--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/okd4o04m7ad65fn4tin87ib4ahbf492m/1688121225000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download&uuid=07de3a47-fcdb-4e53-b97a-19818b579f36
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 74.125.134.132, 2

In [38]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [39]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.007214,0.049746,0.063232,0.213117,0.002548,0.000000,0.002989,0.000000,...,0.000000,0.007045,0.042672,0.098515,0.124398,0.044201,0.060642,0.115349,0.022772,0
1,u108690,0.018730,0.000000,0.000000,0.070961,0.237472,0.004851,0.000000,0.000000,0.004799,...,0.011698,0.007499,0.117693,0.106447,0.113055,0.088997,0.006221,0.095248,0.003433,1
2,u108339,0.090183,0.016986,0.016728,0.137240,0.157661,0.000000,0.004503,0.026918,0.010828,...,0.002067,0.001869,0.104995,0.153738,0.095754,0.038900,0.034494,0.093090,0.000000,1


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [41]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [42]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [43]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.04104892, 0.09064577, 0.56835624, 0.2082399 , 0.18175835,
       0.05229407, 0.07109831, 0.03139962, 0.15470297, 0.06016522])

In [44]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [45]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [46]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.27650696309325196, F-Score=0.653, Precision=0.607, Recall=0.706


In [47]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9364218849933136

In [48]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

<ipython-input-48-67fa86a82b10>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.276507,0.65283,0.607018,0.706122,0.936422


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [49]:
FUNC = np.median

In [50]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.000000,0.014123,0.065195,0.219817,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.015187,0.099337,0.025726,0.012063,0.042975,0.067372,0.0
1,u108690,0.018953,0.000000,0.000000,0.081792,0.258887,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.052754,0.100052,0.108731,0.071286,0.000000,0.047412,0.0
2,u108339,0.119537,0.005815,0.007135,0.122288,0.144636,0.0,0.0,0.006338,0.00966,...,0.0,0.0,0.0,0.080020,0.139003,0.085922,0.030597,0.029504,0.062635,0.0


In [51]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.000000,0.014123,0.065195,0.219817,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.015187,0.099337,0.025726,0.012063,0.042975,0.067372,0.0,0
1,u108690,0.018953,0.000000,0.000000,0.081792,0.258887,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.052754,0.100052,0.108731,0.071286,0.000000,0.047412,0.0,1
2,u108339,0.119537,0.005815,0.007135,0.122288,0.144636,0.0,0.0,0.006338,0.00966,...,0.0,0.0,0.080020,0.139003,0.085922,0.030597,0.029504,0.062635,0.0,1


In [52]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [53]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [54]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.02291414, 0.07149266, 0.49503465, 0.16750665, 0.07492225,
       0.02822478, 0.04414359, 0.01529279, 0.11015544, 0.11762641])

In [55]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.3051982883023916, F-Score=0.695, Precision=0.692, Recall=0.698


In [56]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9523995581138438

In [57]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

<ipython-input-57-67fa86a82b10>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.276507,0.652830,0.607018,0.706122,0.936422
1,median,0.305198,0.695122,0.692308,0.697959,0.952400



3. Повторить п.2, но используя уже не медиану, а max


In [58]:
FUNC = np.max

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.232423,0.069888,0.117489,0.330918,0.000000,0.340741,0.122889,0.032334,0.231983,...,0.276339,0.025991,0.016381,0.366306,0.000000,0.260305,0.128184,0.247024,0.144927,0.300353
1,u108690,0.296334,0.287015,0.016021,0.055022,0.013944,0.048941,0.308129,0.000000,0.104511,...,0.402543,0.083770,0.016419,0.084657,0.054344,0.023330,0.160590,0.409286,0.254788,0.147057
2,u108339,0.044127,0.112746,0.000000,0.347865,0.068541,0.015101,0.073907,0.036893,0.158408,...,0.298438,0.072704,0.034097,0.174184,0.055585,0.152593,0.393400,0.159686,0.230807,0.171170


In [59]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.000000,0.014123,0.065195,0.219817,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.015187,0.099337,0.025726,0.012063,0.042975,0.067372,0.0,0
1,u108690,0.018953,0.000000,0.000000,0.081792,0.258887,0.0,0.0,0.000000,0.00000,...,0.0,0.0,0.052754,0.100052,0.108731,0.071286,0.000000,0.047412,0.0,1
2,u108339,0.119537,0.005815,0.007135,0.122288,0.144636,0.0,0.0,0.006338,0.00966,...,0.0,0.0,0.080020,0.139003,0.085922,0.030597,0.029504,0.062635,0.0,1


In [60]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [61]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [62]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.02291414, 0.07149266, 0.49503465, 0.16750665, 0.07492225,
       0.02822478, 0.04414359, 0.01529279, 0.11015544, 0.11762641])

In [63]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.3051982883023916, F-Score=0.695, Precision=0.692, Recall=0.698


In [64]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9523995581138438

In [65]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

<ipython-input-65-67fa86a82b10>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.276507,0.652830,0.607018,0.706122,0.936422
1,median,0.305198,0.695122,0.692308,0.697959,0.952400
2,amax,0.305198,0.695122,0.692308,0.697959,0.952400


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [68]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [69]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [70]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)

    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [71]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|██████████| 8000/8000 [01:34<00:00, 84.96it/s] 


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.000000,0.121470,0.588437,1.857208,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.141151,0.824047,0.221271,0.112119,0.369625,0.579471,0.0
1,u108690,0.171762,0.000000,0.000000,0.691616,2.160999,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.454324,0.898892,0.897395,0.632731,0.000000,0.404110,0.0
2,u108339,0.964763,0.054049,0.061368,1.000738,1.174969,0.0,0.0,0.058909,0.075252,...,0.0,0.0,0.0,0.713797,1.185015,0.728508,0.267843,0.248846,0.525221,0.0


In [72]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.000000,0.121470,0.588437,1.857208,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.141151,0.824047,0.221271,0.112119,0.369625,0.579471,0.0,0
1,u108690,0.171762,0.000000,0.000000,0.691616,2.160999,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.454324,0.898892,0.897395,0.632731,0.000000,0.404110,0.0,1
2,u108339,0.964763,0.054049,0.061368,1.000738,1.174969,0.0,0.0,0.058909,0.075252,...,0.0,0.0,0.713797,1.185015,0.728508,0.267843,0.248846,0.525221,0.0,1


In [73]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]],
                                                    X['churn'], random_state=0)

In [74]:
model = LogisticRegression()
# обучим
model.fit(X_train, y_train)

LogisticRegression()

In [75]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([4.11461372e-04, 1.06843815e-03, 6.91089757e-01, 1.08864745e-01,
       2.40573195e-03, 7.22477937e-04, 5.96790297e-03, 6.65260244e-06,
       2.65903463e-02, 1.10274476e-01])

In [76]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.41925050240345807, F-Score=0.788, Precision=0.810, Recall=0.767


In [77]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.974891563462992

In [78]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

<ipython-input-78-18b95ddf6d03>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({



5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [79]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.276507,0.652830,0.607018,0.706122,0.936422
1,median,0.305198,0.695122,0.692308,0.697959,0.952400
2,amax,0.305198,0.695122,0.692308,0.697959,0.952400
3,idf_median,0.419251,0.788260,0.810345,0.767347,0.974892


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

Исходя из полученного результата можно сделать вывод что метод idf_median оказался эффективнее всего.